In [1]:
import json
import pandas as pd

def json_to_dataframe(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    df = pd.json_normalize(data)
    return df

# from https://www.sec.gov/edgar/sec-api-documentation
json_to_dataframe("companyfacts_sample.json")

,cik,entityName,facts.dei.EntityCommonStockSharesOutstanding.label,facts.dei.EntityCommonStockSharesOutstanding.description,facts.dei.EntityCommonStockSharesOutstanding.units.shares,facts.dei.EntityPublicFloat.label,facts.dei.EntityPublicFloat.description,facts.dei.EntityPublicFloat.units.USD,facts.us-gaap.AccountsPayableCurrent.label,facts.us-gaap.AccountsPayableCurrent.description,...,facts.us-gaap.LossContingencyDamagesSoughtValue.units.USD,facts.us-gaap.RestrictedInvestmentsCurrent.label,facts.us-gaap.RestrictedInvestmentsCurrent.description,facts.us-gaap.RestrictedInvestmentsCurrent.units.USD,facts.us-gaap.DefinedBenefitPlanFairValueOfPlanAssets.label,facts.us-gaap.DefinedBenefitPlanFairValueOfPlanAssets.description,facts.us-gaap.DefinedBenefitPlanFairValueOfPlanAssets.units.USD,facts.us-gaap.DefinedBenefitPlanAssetsForPlanBenefitsNoncurrent.label,facts.us-gaap.DefinedBenefitPlanAssetsForPlanBenefitsNoncurrent.description,facts.us-gaap.DefinedBenefitPlanAssetsForPlanBenefitsNoncurrent.units.USD
0,1750,AAR CORP,"Entity Common Stock, Shares Outstanding",Indicate number of shares or other units outst...,"[{'end': '2010-08-31', 'val': 39662816, 'accn'...",Entity Public Float,The aggregate market value of the voting and n...,"[{'end': '2010-11-30', 'val': 934567976, 'accn...","Accounts Payable, Current",Carrying value as of the balance sheet date of...,...,"[{'start': '2022-03-01', 'end': '2022-03-31', ...","Restricted Investments, Current",This element represents the current portion of...,"[{'end': '2023-05-31', 'val': 1300000, 'accn':...","Defined Benefit Plan, Plan Assets, Amount",Amount of asset segregated and restricted to p...,"[{'end': '2023-08-31', 'val': 7600000, 'accn':...","Assets for Plan Benefits, Defined Benefit Plan","Amount of asset, recognized in statement of fi...","[{'end': '2022-11-30', 'val': 6900000, 'accn':..."


In [2]:

import json
import pandas as pd

# JSON data as a multi-line string
json_data = """
{
    "cik": 1750,
    "entityName": "AAR CORP",
    "facts": {
        "us-gaap": {
            "AccountsPayableCurrent": {
                "label": "Accounts Payable, Current",
                "description": "Carrying value as of the balance sheet date of liabilities incurred (and for which invoices have typically been received) and payable to vendors for goods and services received that are used in an entity's business. Used to reflect the current portion of the liabilities (due within one year or within the normal operating cycle if longer).",
                "units": {
                    "USD": [
                        {
                            "end": "2010-05-31",
                            "val": 114906000,
                            "accn": "0001104659-10-049632",
                            "fy": 2011,
                            "fp": "Q1",
                            "form": "10-Q",
                            "filed": "2010-09-23"
                        }
                    ]
                }
            },
            "AccountsReceivableNetCurrent": {
                "label": "Accounts Receivable, after Allowance for Credit Loss, Current",
                "description": "Amount, after allowance for credit loss, of right to consideration from customer for product sold and service rendered in normal course of business, classified as current.",
                "units": {
                    "USD": [
                        {
                            "end": "2010-05-31",
                            "val": 238466000,
                            "accn": "0001104659-10-049632",
                            "fy": 2011,
                            "fp": "Q1",
                            "form": "10-Q",
                            "filed": "2010-09-23"
                        }
                    ]
                }
            }
        }
    }
}
"""

# Parse the JSON data
data = json.loads(json_data)

# Iterate over each item in 'us-gaap' and create a DataFrame
all_data_frames = []
for key, value in data['facts']['us-gaap'].items():
    temp_df = pd.DataFrame(value['units']['USD'])
    temp_df['fact'] = key  # Add a column to identify the fact type
    all_data_frames.append(temp_df)

# Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(all_data_frames, ignore_index=True)
final_df

,end,val,accn,fy,fp,form,filed,fact
0,2010-05-31,114906000,0001104659-10-049632,2011,Q1,10-Q,2010-09-23,AccountsPayableCurrent
1,2010-05-31,238466000,0001104659-10-049632,2011,Q1,10-Q,2010-09-23,AccountsReceivableNetCurrent


In [3]:


def json_to_dataframe(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)
        # Create an empty list to store the rows
        rows = []
        # Iterate over each item in 'us-gaap' and create a row for each currency type for each fact type
        for fact, fact_info in data['facts']['us-gaap'].items():
            for currency, samples in fact_info['units'].items():
                for sample in samples:
                    if sample['fp'].startswith('Q'):
                        # Create a dictionary for the row
                        row = {
                            'entity_name': data['entityName'],
                            'currency': currency,
                            'label': fact_info['label'],
                            'description': fact_info['description'],
                            'fact': fact,
                        }
    
                        # Add the currency_info values to the row dictionary
                        row.update(sample)
                        # Append the row to the list
                        rows.append(row)
                        
        # Create a dataframe from the list of rows
        df = pd.DataFrame(rows)
        df['period_date'] = pd.to_datetime(df['fy'].astype(str) + '-' + df['fp']) 
        return df

company_fact_df = json_to_dataframe("companyfacts_sample.json")
company_fact_df

C:\Users\darrr\AppData\Local\Temp\ipykernel_14144\4263286305.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['period_date'] = pd.to_datetime(df['fy'].astype(str) + '-' + df['fp'])


,entity_name,currency,label,description,fact,end,val,accn,fy,fp,form,filed,frame,start,period_date
0,AAR CORP,USD,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,AccountsPayableCurrent,2010-05-31,114906000.0,0001104659-10-049632,2011,Q1,10-Q,2010-09-23,NaN,NaN,2011-01-01
1,AAR CORP,USD,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,AccountsPayableCurrent,2010-05-31,114906000.0,0001104659-10-063683,2011,Q2,10-Q,2010-12-21,NaN,NaN,2011-04-01
2,AAR CORP,USD,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,AccountsPayableCurrent,2010-05-31,114906000.0,0001104659-11-015691,2011,Q3,10-Q,2011-03-22,NaN,NaN,2011-07-01
3,AAR CORP,USD,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,AccountsPayableCurrent,2010-08-31,130112000.0,0001104659-10-049632,2011,Q1,10-Q,2010-09-23,CY2010Q3I,NaN,2011-01-01
4,AAR CORP,USD,"Accounts Payable, Current",Carrying value as of the balance sheet date of...,AccountsPayableCurrent,2010-11-30,129789000.0,0001104659-10-063683,2011,Q2,10-Q,2010-12-21,CY2010Q4I,NaN,2011-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11421,AAR CORP,USD,"Cash, Cash Equivalents, Restricted Cash and Re...","Amount of cash and cash equivalents, and cash ...",CashCashEquivalentsRestrictedCashAndRestricted...,2023-11-30,75500000.0,0001104659-23-128321,2024,Q2,10-Q,2023-12-21,CY2023Q4I,NaN,2024-04-01
11422,AAR CORP,USD,"Loss Contingency, Damages Sought, Value",The value (monetary amount) of the award the p...,LossContingencyDamagesSoughtValue,2023-03-31,1800000.0,0001104659-23-103944,2023,Q1,10-Q,2023-09-26,NaN,2023-03-01,2023-01-01
11423,AAR CORP,USD,"Loss Contingency, Damages Sought, Value",The value (monetary amount) of the award the p...,LossContingencyDamagesSoughtValue,2023-03-31,1800000.0,0001104659-23-128321,2024,Q2,10-Q,2023-12-21,CY2023Q1I,2023-03-01,2024-04-01
11424,AAR CORP,USD,"Defined Benefit Plan, Plan Assets, Amount",Amount of asset segregated and restricted to p...,DefinedBenefitPlanFairValueOfPlanAssets,2023-08-31,7600000.0,0001104659-23-103944,2023,Q1,10-Q,2023-09-26,CY2023Q3I,NaN,2023-01-01


In [4]:
company_fact_df.describe()

,val,fy,period_date
count,1.142600e+04,11426.000000,11426
mean,1.260138e+08,2017.660949,2017-12-06 03:04:30.330824704
min,-4.050000e+08,2011.000000,2011-01-01 00:00:00
25%,1.460000e+05,2014.000000,2014-07-01 00:00:00
50%,1.234000e+07,2018.000000,2018-04-01 00:00:00
75%,8.477500e+07,2021.000000,2021-04-01 00:00:00
max,2.227700e+09,2024.000000,2024-04-01 00:00:00
std,2.989213e+08,3.811746,NaN


In [5]:
import math
import random
import plotly.graph_objects as go

color_codes = {}
def stringToColorCode(str) :
    if str in color_codes:
        return color_codes[str] 
    else:
        color_codes[str] = '#' + ('000000' + format(math.floor(random.random() * 16777215), '06x'))[-6:], len(str)
    return color_codes

def render_fact(df_input):
    
    if 'revenue' not in str.lower(df_input['fact'].iloc[0]):
    # if df_input['fact'].iloc[0] not in ['EarningsPerShareBasic']: #, 'EarningsPerShareDiluted'] :
        return False
    
    df_copy = df_input.copy(deep=True)  # Make a copy of the dataframe
    df_copy.reset_index(drop=True, inplace=True)
    df_copy.set_index('end', inplace=True)  # Set 'end' column as the new index
    df_copy.sort_values(by='end', inplace=True)
    # df_copy.dropna(inplace=True)  # Drop rows with missing values
    display(df_copy)
    
    fig = go.FigureWidget()
    fig.add_trace(
        go.Scatter(
            x=list(df_copy.index),
            y=list(df_copy.val),
            name=df_copy['fact'].iloc[0],

            
            #line=dict(color=stringToColorCode(df_copy['fact'].iloc[0])),
            showlegend=True
        )
    )
    
    fig.update_xaxes(title="end", rangeslider_visible=True)
    fig.update_yaxes()  # Fit y-axis to match the current slider view data)

    fig.update_layout(title=df_copy['entity_name'].iloc[0] + ' ' + df_copy['fact'].iloc[0],
        height=800, width=1200, showlegend=True)
        
    fig.show()

    # # fig = go.FigureWidget()
    # for column in df_copy.columns() : 
    #     fig = go.FigureWidget()
    #     # plt.plot(df_copy.index, df_copy[column], label=column)  # Plot each column
       
    #     fig.add_trace(
    #         go.Scatter(
    #             x=list(df_copy.index),
    #             y=list(df_copy[column]),
    #             name=column,
    #             line=dict(color=stringToColorCode(column))))
    #     fig.show()

# FILEPATH: /c:/src/market/stocks-intro.ipynb

company_fact_df.groupby(by='fact').apply(render_fact)  # add all metrics to the subset of the dataframe



,entity_name,currency,label,description,fact,val,accn,fy,fp,form,filed,frame,start,period_date
end,,,,,,,,,,,,,,
2018-08-31,AAR CORP,USD,"Contract with Customer, Asset, Cumulative Catc...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerAssetCumulativeCatchUpAdju...,700000.0,0001410578-19-001436,2020,Q1,10-Q,2019-09-26,CY2018Q3,2018-06-01,2020-01-01
2018-11-30,AAR CORP,USD,"Contract with Customer, Asset, Cumulative Catc...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerAssetCumulativeCatchUpAdju...,3800000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,NaN,2018-06-01,2020-04-01
2018-11-30,AAR CORP,USD,"Contract with Customer, Asset, Cumulative Catc...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerAssetCumulativeCatchUpAdju...,3100000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,CY2018Q4,2018-09-01,2020-04-01
2019-02-28,AAR CORP,USD,"Contract with Customer, Asset, Cumulative Catc...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerAssetCumulativeCatchUpAdju...,4600000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,NaN,2018-06-01,2020-07-01
2019-02-28,AAR CORP,USD,"Contract with Customer, Asset, Cumulative Catc...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerAssetCumulativeCatchUpAdju...,800000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,CY2019Q1,2018-12-01,2020-07-01
2019-11-30,AAR CORP,USD,"Contract with Customer, Asset, Cumulative Catc...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerAssetCumulativeCatchUpAdju...,1900000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,NaN,2019-06-01,2020-04-01
2019-11-30,AAR CORP,USD,"Contract with Customer, Asset, Cumulative Catc...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerAssetCumulativeCatchUpAdju...,1900000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,CY2019Q4,2019-09-01,2020-04-01
2020-02-29,AAR CORP,USD,"Contract with Customer, Asset, Cumulative Catc...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerAssetCumulativeCatchUpAdju...,6100000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,NaN,2019-06-01,2020-07-01
2020-02-29,AAR CORP,USD,"Contract with Customer, Asset, Cumulative Catc...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerAssetCumulativeCatchUpAdju...,4400000.0,0001104659-21-040394,2021,Q3,10-Q,2021-03-23,NaN,2019-06-01,2021-07-01


,entity_name,currency,label,description,fact,val,accn,fy,fp,form,filed,frame,start,period_date
end,,,,,,,,,,,,,,
2018-08-31,AAR CORP,USD,"Contract with Customer, Liability, Cumulative ...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerLiabilityCumulativeCatchUp...,500000.0,0001410578-19-001436,2020,Q1,10-Q,2019-09-26,CY2018Q3,2018-06-01,2020-01-01
2018-11-30,AAR CORP,USD,"Contract with Customer, Liability, Cumulative ...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerLiabilityCumulativeCatchUp...,500000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,NaN,2018-06-01,2020-04-01
2019-02-28,AAR CORP,USD,"Contract with Customer, Liability, Cumulative ...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerLiabilityCumulativeCatchUp...,500000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,NaN,2018-06-01,2020-07-01
2020-02-29,AAR CORP,USD,"Contract with Customer, Liability, Cumulative ...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerLiabilityCumulativeCatchUp...,1700000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,NaN,2019-06-01,2020-07-01
2020-02-29,AAR CORP,USD,"Contract with Customer, Liability, Cumulative ...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerLiabilityCumulativeCatchUp...,1700000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,CY2020Q1,2019-12-01,2020-07-01
2021-08-31,AAR CORP,USD,"Contract with Customer, Liability, Cumulative ...",Amount of increase (decrease) in revenue recog...,ContractWithCustomerLiabilityCumulativeCatchUp...,-1000000.0,0001104659-22-102374,2023,Q1,10-Q,2022-09-22,CY2021Q3,2021-06-01,2023-01-01


,entity_name,currency,label,description,fact,val,accn,fy,fp,form,filed,frame,start,period_date
end,,,,,,,,,,,,,,
2018-11-30,AAR CORP,USD,"Contract with Customer, Liability, Revenue Rec...",Amount of revenue recognized that was previous...,ContractWithCustomerLiabilityRevenueRecognized,156500000.0,0001104659-18-073842,2019,Q2,10-Q,2018-12-19,NaN,2018-06-01,2019-04-01
2018-11-30,AAR CORP,USD,"Contract with Customer, Liability, Revenue Rec...",Amount of revenue recognized that was previous...,ContractWithCustomerLiabilityRevenueRecognized,156500000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,NaN,2018-06-01,2020-04-01
2018-11-30,AAR CORP,USD,"Contract with Customer, Liability, Revenue Rec...",Amount of revenue recognized that was previous...,ContractWithCustomerLiabilityRevenueRecognized,76400000.0,0001104659-18-073842,2019,Q2,10-Q,2018-12-19,NaN,2018-09-01,2019-04-01
2018-11-30,AAR CORP,USD,"Contract with Customer, Liability, Revenue Rec...",Amount of revenue recognized that was previous...,ContractWithCustomerLiabilityRevenueRecognized,76400000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,CY2018Q4,2018-09-01,2020-04-01
2019-02-28,AAR CORP,USD,"Contract with Customer, Liability, Revenue Rec...",Amount of revenue recognized that was previous...,ContractWithCustomerLiabilityRevenueRecognized,247800000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,NaN,2018-06-01,2020-07-01
2019-02-28,AAR CORP,USD,"Contract with Customer, Liability, Revenue Rec...",Amount of revenue recognized that was previous...,ContractWithCustomerLiabilityRevenueRecognized,91300000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,CY2019Q1,2018-12-01,2020-07-01
2019-11-30,AAR CORP,USD,"Contract with Customer, Liability, Revenue Rec...",Amount of revenue recognized that was previous...,ContractWithCustomerLiabilityRevenueRecognized,204300000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,NaN,2019-06-01,2020-04-01
2019-11-30,AAR CORP,USD,"Contract with Customer, Liability, Revenue Rec...",Amount of revenue recognized that was previous...,ContractWithCustomerLiabilityRevenueRecognized,103300000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,CY2019Q4,2019-09-01,2020-04-01
2020-02-29,AAR CORP,USD,"Contract with Customer, Liability, Revenue Rec...",Amount of revenue recognized that was previous...,ContractWithCustomerLiabilityRevenueRecognized,326000000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,NaN,2019-06-01,2020-07-01


,entity_name,currency,label,description,fact,val,accn,fy,fp,form,filed,frame,start,period_date
end,,,,,,,,,,,,,,
2019-05-31,AAR CORP,USD,"Deferred Revenue, Current",Amount of deferred income and obligation to tr...,DeferredRevenueCurrent,12600000.0,0001410578-19-001436,2020,Q1,10-Q,2019-09-26,NaN,NaN,2020-01-01
2019-05-31,AAR CORP,USD,"Deferred Revenue, Current",Amount of deferred income and obligation to tr...,DeferredRevenueCurrent,12600000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,NaN,NaN,2020-04-01
2019-05-31,AAR CORP,USD,"Deferred Revenue, Current",Amount of deferred income and obligation to tr...,DeferredRevenueCurrent,12600000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,NaN,NaN,2020-07-01
2019-08-31,AAR CORP,USD,"Deferred Revenue, Current",Amount of deferred income and obligation to tr...,DeferredRevenueCurrent,10700000.0,0001410578-19-001436,2020,Q1,10-Q,2019-09-26,CY2019Q3I,NaN,2020-01-01
2019-11-30,AAR CORP,USD,"Deferred Revenue, Current",Amount of deferred income and obligation to tr...,DeferredRevenueCurrent,11300000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,CY2019Q4I,NaN,2020-04-01
2020-02-29,AAR CORP,USD,"Deferred Revenue, Current",Amount of deferred income and obligation to tr...,DeferredRevenueCurrent,13400000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,CY2020Q1I,NaN,2020-07-01
2020-05-31,AAR CORP,USD,"Deferred Revenue, Current",Amount of deferred income and obligation to tr...,DeferredRevenueCurrent,11200000.0,0001104659-20-108360,2021,Q1,10-Q,2020-09-24,NaN,NaN,2021-01-01
2020-05-31,AAR CORP,USD,"Deferred Revenue, Current",Amount of deferred income and obligation to tr...,DeferredRevenueCurrent,11200000.0,0001104659-20-137498,2021,Q2,10-Q,2020-12-18,NaN,NaN,2021-04-01
2020-05-31,AAR CORP,USD,"Deferred Revenue, Current",Amount of deferred income and obligation to tr...,DeferredRevenueCurrent,11200000.0,0001104659-21-040394,2021,Q3,10-Q,2021-03-23,CY2020Q2I,NaN,2021-07-01


,entity_name,currency,label,description,fact,val,accn,fy,fp,form,filed,frame,start,period_date
end,,,,,,,,,,,,,,
2018-05-31,AAR CORP,USD,"Deferred Revenue, Noncurrent",Amount of deferred income and obligation to tr...,DeferredRevenueNoncurrent,35800000.0,0001104659-19-016320,2019,Q3,10-Q,2019-03-20,CY2018Q2I,NaN,2019-07-01
2019-02-28,AAR CORP,USD,"Deferred Revenue, Noncurrent",Amount of deferred income and obligation to tr...,DeferredRevenueNoncurrent,89000000.0,0001104659-19-016320,2019,Q3,10-Q,2019-03-20,CY2019Q1I,NaN,2019-07-01
2019-05-31,AAR CORP,USD,"Deferred Revenue, Noncurrent",Amount of deferred income and obligation to tr...,DeferredRevenueNoncurrent,83800000.0,0001410578-19-001436,2020,Q1,10-Q,2019-09-26,NaN,NaN,2020-01-01
2019-05-31,AAR CORP,USD,"Deferred Revenue, Noncurrent",Amount of deferred income and obligation to tr...,DeferredRevenueNoncurrent,83800000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,NaN,NaN,2020-04-01
2019-05-31,AAR CORP,USD,"Deferred Revenue, Noncurrent",Amount of deferred income and obligation to tr...,DeferredRevenueNoncurrent,83800000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,NaN,NaN,2020-07-01
2019-08-31,AAR CORP,USD,"Deferred Revenue, Noncurrent",Amount of deferred income and obligation to tr...,DeferredRevenueNoncurrent,75600000.0,0001410578-19-001436,2020,Q1,10-Q,2019-09-26,CY2019Q3I,NaN,2020-01-01
2019-11-30,AAR CORP,USD,"Deferred Revenue, Noncurrent",Amount of deferred income and obligation to tr...,DeferredRevenueNoncurrent,122500000.0,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,CY2019Q4I,NaN,2020-04-01
2020-02-29,AAR CORP,USD,"Deferred Revenue, Noncurrent",Amount of deferred income and obligation to tr...,DeferredRevenueNoncurrent,101300000.0,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,CY2020Q1I,NaN,2020-07-01
2020-05-31,AAR CORP,USD,"Deferred Revenue, Noncurrent",Amount of deferred income and obligation to tr...,DeferredRevenueNoncurrent,88000000.0,0001104659-20-108360,2021,Q1,10-Q,2020-09-24,NaN,NaN,2021-01-01


...

,entity_name,currency,label,description,fact,val,accn,fy,fp,form,filed,frame,start,period_date
end,,,,,,,,,,,,,,
2018-08-31,AAR CORP,pure,"Revenue, Remaining Performance Obligation, Per...",Percentage of remaining performance obligation...,RevenueRemainingPerformanceObligationPercentage,0.5,0001104659-18-058911,2019,Q1,10-Q,2018-09-27,CY2018Q3I,NaN,2019-01-01
2018-11-30,AAR CORP,pure,"Revenue, Remaining Performance Obligation, Per...",Percentage of remaining performance obligation...,RevenueRemainingPerformanceObligationPercentage,0.4,0001104659-18-073842,2019,Q2,10-Q,2018-12-19,CY2018Q4I,NaN,2019-04-01
2019-02-28,AAR CORP,pure,"Revenue, Remaining Performance Obligation, Per...",Percentage of remaining performance obligation...,RevenueRemainingPerformanceObligationPercentage,0.4,0001104659-19-016320,2019,Q3,10-Q,2019-03-20,CY2019Q1I,NaN,2019-07-01
2019-08-31,AAR CORP,pure,"Revenue, Remaining Performance Obligation, Per...",Percentage of remaining performance obligation...,RevenueRemainingPerformanceObligationPercentage,0.4,0001410578-19-001436,2020,Q1,10-Q,2019-09-26,CY2019Q3I,NaN,2020-01-01
2019-11-30,AAR CORP,pure,"Revenue, Remaining Performance Obligation, Per...",Percentage of remaining performance obligation...,RevenueRemainingPerformanceObligationPercentage,0.4,0001104659-19-074491,2020,Q2,10-Q,2019-12-20,CY2019Q4I,NaN,2020-04-01
2020-02-29,AAR CORP,pure,"Revenue, Remaining Performance Obligation, Per...",Percentage of remaining performance obligation...,RevenueRemainingPerformanceObligationPercentage,0.4,0001104659-20-037857,2020,Q3,10-Q,2020-03-25,CY2020Q1I,NaN,2020-07-01


,entity_name,currency,label,description,fact,val,accn,fy,fp,form,filed,frame,start,period_date
end,,,,,,,,,,,,,,
2009-08-31,AAR CORP,USD,"Sales Revenue, Goods, Net (Deprecated 2018-01-31)",Aggregate revenue during the period from the s...,SalesRevenueGoodsNet,272529000.0,0001104659-10-049632,2011,Q1,10-Q,2010-09-23,CY2009Q3,2009-06-01,2011-01-01
2009-11-30,AAR CORP,USD,"Sales Revenue, Goods, Net (Deprecated 2018-01-31)",Aggregate revenue during the period from the s...,SalesRevenueGoodsNet,541875000.0,0001104659-10-063683,2011,Q2,10-Q,2010-12-21,NaN,2009-06-01,2011-04-01
2009-11-30,AAR CORP,USD,"Sales Revenue, Goods, Net (Deprecated 2018-01-31)",Aggregate revenue during the period from the s...,SalesRevenueGoodsNet,269346000.0,0001104659-10-063683,2011,Q2,10-Q,2010-12-21,CY2009Q4,2009-09-01,2011-04-01
2010-02-28,AAR CORP,USD,"Sales Revenue, Goods, Net (Deprecated 2018-01-31)",Aggregate revenue during the period from the s...,SalesRevenueGoodsNet,771196000.0,0001104659-11-015691,2011,Q3,10-Q,2011-03-22,NaN,2009-06-01,2011-07-01
2010-02-28,AAR CORP,USD,"Sales Revenue, Goods, Net (Deprecated 2018-01-31)",Aggregate revenue during the period from the s...,SalesRevenueGoodsNet,243815000.0,0001104659-11-015691,2011,Q3,10-Q,2011-03-22,CY2010Q1,2009-12-01,2011-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-31,AAR CORP,USD,"Sales Revenue, Goods, Net (Deprecated 2018-01-31)",Aggregate revenue during the period from the s...,SalesRevenueGoodsNet,244700000.0,0001104659-17-058003,2018,Q1,10-Q,2017-09-20,CY2017Q3,2017-06-01,2018-01-01
2017-11-30,AAR CORP,USD,"Sales Revenue, Goods, Net (Deprecated 2018-01-31)",Aggregate revenue during the period from the s...,SalesRevenueGoodsNet,498600000.0,0001104659-17-074650,2018,Q2,10-Q,2017-12-22,NaN,2017-06-01,2018-04-01
2017-11-30,AAR CORP,USD,"Sales Revenue, Goods, Net (Deprecated 2018-01-31)",Aggregate revenue during the period from the s...,SalesRevenueGoodsNet,253900000.0,0001104659-17-074650,2018,Q2,10-Q,2017-12-22,CY2017Q4,2017-09-01,2018-04-01


,entity_name,currency,label,description,fact,val,accn,fy,fp,form,filed,frame,start,period_date
end,,,,,,,,,,,,,,
2009-08-31,AAR CORP,USD,"Revenue, Net (Deprecated 2018-01-31)",Total revenue from sale of goods and services ...,SalesRevenueNet,3.415230e+08,0001104659-10-049632,2011,Q1,10-Q,2010-09-23,CY2009Q3,2009-06-01,2011-01-01
2009-11-30,AAR CORP,USD,"Revenue, Net (Deprecated 2018-01-31)",Total revenue from sale of goods and services ...,SalesRevenueNet,6.702070e+08,0001104659-10-063683,2011,Q2,10-Q,2010-12-21,NaN,2009-06-01,2011-04-01
2009-11-30,AAR CORP,USD,"Revenue, Net (Deprecated 2018-01-31)",Total revenue from sale of goods and services ...,SalesRevenueNet,3.286840e+08,0001104659-10-063683,2011,Q2,10-Q,2010-12-21,CY2009Q4,2009-09-01,2011-04-01
2010-02-28,AAR CORP,USD,"Revenue, Net (Deprecated 2018-01-31)",Total revenue from sale of goods and services ...,SalesRevenueNet,9.516070e+08,0001104659-11-015691,2011,Q3,10-Q,2011-03-22,NaN,2009-06-01,2011-07-01
2010-02-28,AAR CORP,USD,"Revenue, Net (Deprecated 2018-01-31)",Total revenue from sale of goods and services ...,SalesRevenueNet,3.008450e+08,0001104659-11-015691,2011,Q3,10-Q,2011-03-22,CY2010Q1,2009-12-01,2011-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-31,AAR CORP,USD,"Revenue, Net (Deprecated 2018-01-31)",Total revenue from sale of goods and services ...,SalesRevenueNet,4.392000e+08,0001104659-17-058003,2018,Q1,10-Q,2017-09-20,NaN,2017-06-01,2018-01-01
2017-11-30,AAR CORP,USD,"Revenue, Net (Deprecated 2018-01-31)",Total revenue from sale of goods and services ...,SalesRevenueNet,8.889000e+08,0001104659-17-074650,2018,Q2,10-Q,2017-12-22,NaN,2017-06-01,2018-04-01
2017-11-30,AAR CORP,USD,"Revenue, Net (Deprecated 2018-01-31)",Total revenue from sale of goods and services ...,SalesRevenueNet,4.497000e+08,0001104659-17-074650,2018,Q2,10-Q,2017-12-22,NaN,2017-09-01,2018-04-01


,entity_name,currency,label,description,fact,val,accn,fy,fp,form,filed,frame,start,period_date
end,,,,,,,,,,,,,,
2009-08-31,AAR CORP,USD,"Sales Revenue, Services, Net (Deprecated 2018-...",Aggregate revenue during the period from servi...,SalesRevenueServicesNet,68994000.0,0001104659-10-049632,2011,Q1,10-Q,2010-09-23,CY2009Q3,2009-06-01,2011-01-01
2009-11-30,AAR CORP,USD,"Sales Revenue, Services, Net (Deprecated 2018-...",Aggregate revenue during the period from servi...,SalesRevenueServicesNet,128332000.0,0001104659-10-063683,2011,Q2,10-Q,2010-12-21,NaN,2009-06-01,2011-04-01
2009-11-30,AAR CORP,USD,"Sales Revenue, Services, Net (Deprecated 2018-...",Aggregate revenue during the period from servi...,SalesRevenueServicesNet,59338000.0,0001104659-10-063683,2011,Q2,10-Q,2010-12-21,CY2009Q4,2009-09-01,2011-04-01
2010-02-28,AAR CORP,USD,"Sales Revenue, Services, Net (Deprecated 2018-...",Aggregate revenue during the period from servi...,SalesRevenueServicesNet,180411000.0,0001104659-11-015691,2011,Q3,10-Q,2011-03-22,NaN,2009-06-01,2011-07-01
2010-02-28,AAR CORP,USD,"Sales Revenue, Services, Net (Deprecated 2018-...",Aggregate revenue during the period from servi...,SalesRevenueServicesNet,57030000.0,0001104659-11-015691,2011,Q3,10-Q,2011-03-22,CY2010Q1,2009-12-01,2011-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-31,AAR CORP,USD,"Sales Revenue, Services, Net (Deprecated 2018-...",Aggregate revenue during the period from servi...,SalesRevenueServicesNet,194500000.0,0001104659-17-058003,2018,Q1,10-Q,2017-09-20,CY2017Q3,2017-06-01,2018-01-01
2017-11-30,AAR CORP,USD,"Sales Revenue, Services, Net (Deprecated 2018-...",Aggregate revenue during the period from servi...,SalesRevenueServicesNet,390300000.0,0001104659-17-074650,2018,Q2,10-Q,2017-12-22,NaN,2017-06-01,2018-04-01
2017-11-30,AAR CORP,USD,"Sales Revenue, Services, Net (Deprecated 2018-...",Aggregate revenue during the period from servi...,SalesRevenueServicesNet,195800000.0,0001104659-17-074650,2018,Q2,10-Q,2017-12-22,CY2017Q4,2017-09-01,2018-04-01


fact
AccountsPayableCurrent                                                      False
AccountsReceivableNetCurrent                                                False
AccruedLiabilitiesCurrent                                                   False
AccruedLiabilitiesCurrentAndNoncurrent                                      False
AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment    False
                                                                            ...  
UndistributedEarningsLossAllocatedToParticipatingSecuritiesBasic            False
UndistributedEarningsLossAllocatedToParticipatingSecuritiesDiluted          False
VariableLeaseCost                                                           False
WeightedAverageNumberOfDilutedSharesOutstanding                             False
WeightedAverageNumberOfSharesOutstandingBasic                               False
Length: 306, dtype: object

In [9]:

import json
import pandas as pd
import plotly.graph_objects as go
import requests
import logging

# Initialize logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

desired_tickers = ['QRTEA', 'AAPL', 'MSFT', 'AMZN', 'GOOG', 'SCHD', 'FREL', 'QYLD', 'NUSI', 'VYM', 'VIG', 'VTI', 'VOO', 'DIA', 'QQQ', 'SPY']
# create request header
headers = {'User-Agent': "rockdeity@gmail.com"}

# get all companies data
companyTickers = requests.get(
    "https://www.sec.gov/files/company_tickers.json",
    headers=headers
)

# review response / keys
# logger.info(list(companyTickers.json().keys()))

# format response to dictionary and get first key/value
firstEntry = companyTickers.json()['0']

# parse CIK // without leading zeros
directCik = companyTickers.json()['0']['cik_str']

ticker = companyTickers.json()['0']['ticker']

# dictionary to dataframe
companyData = pd.DataFrame.from_dict(companyTickers.json(), orient='index')

# add leading zeros to CIK
companyData['cik_str'] = companyData['cik_str'].astype(str).str.zfill(10)

companyData



,cik_str,ticker,title
0,0000320193,AAPL,Apple Inc.
1,0000789019,MSFT,MICROSOFT CORP
2,0001652044,GOOGL,Alphabet Inc.
3,0001018724,AMZN,AMAZON COM INC
4,0001045810,NVDA,NVIDIA CORP
...,...,...,...
10638,0001953530,OCSAW,Oculis Holding AG
10639,0001978528,QETAU,Quetta Acquisition Corp
10640,0001978528,QETAR,Quetta Acquisition Corp
10641,0001935418,FMSTW,Foremost Lithium Resource & Technology Ltd.


In [15]:


# cik = companyData[0:1].cik_str[0]

def process_cik_data(cik):
    # review data
    logger.info(f"Processing CIK: {cik}")

    # get company specific filing metadata
    filingMetadata = requests.get(
        f'https://data.sec.gov/submissions/CIK{cik}.json',
        headers=headers
    )

    # review json
    # logger.info(f"filingMetadata.json(): {json.dumps(filingMetadata.json(), indent=4)}")
    # filingMetadata.json()['filings']
    # filingMetadata.json()['filings'].keys()
    # filingMetadata.json()['filings']['recent']
    # logger.info(f"filingMetadata.json()['filings']['recent']: {json.dumps(filingMetadata.json()['filings']['recent'], indent=4)}")
    # filingMetadata.json()['filings']['recent'].keys()

    # dictionary to dataframe
    allForms = pd.DataFrame.from_dict(filingMetadata.json()['filings']['recent'])

    # review columns
    # allForms.columns
    # allForms[['accessionNumber', 'reportDate', 'form']].head(50)
    # display(allForms.head(12))

    # 10-Q metadata
    # display(allForms.where(allForms['form'] == '10-K').head())

    # get company facts data
    companyFacts = requests.get(
        f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json',
        headers=headers
    )

    # review data
    # logger.info(f"companyFacts.json(): {json.dumps(companyFacts.json(), indent=4)}")
    # companyFacts.json().keys()
    # companyFacts.json()['facts']
    # companyFacts.json()['facts'].keys()

    # filing metadata
    # companyFacts.json()['facts']['dei']['EntityCommonStockSharesOutstanding']
    # companyFacts.json()['facts']['dei']['EntityCommonStockSharesOutstanding'].keys()
    # companyFacts.json()['facts']['dei']['EntityCommonStockSharesOutstanding']['units']
    # companyFacts.json()['facts']['dei']['EntityCommonStockSharesOutstanding']['units']['shares']
    # companyFacts.json()['facts']['dei']['EntityCommonStockSharesOutstanding']['units']['shares'][0]

    # concept data // financial statement line items
    # companyFacts.json()['facts']['us-gaap']
    # companyFacts.json()['facts']['us-gaap'].keys()
    # fact_keys = '\n'.join(list(companyFacts.json()['facts']['us-gaap'].keys()))
    # logger.info(f"companyFacts.json()['facts']['us-gaap'].keys(): \n {fact_keys}")

    # different amounts of data available per concept
    # companyFacts.json()['facts']['us-gaap']['AccountsPayable']
    # companyFacts.json()['facts']['us-gaap']['Revenues']
    # companyFacts.json()['facts']['us-gaap']['Assets']

    all_facts = companyFacts.json()['facts']['us-gaap']
    # display(all_facts)
    graph_facts = [
        # 'Revenues', 
        'Assets', 
        # 'Dividends', 
        'EarningsPerShareBasic',
        # 'AccruedLiabilities',
        # 'AccruedMarketingCostsCurrent',
        # 'AdvertisingExpense', 
        # 'AvailableForSaleSecuritiesCurrent',
        'Cash',
        # 'CommonStockValue',
        # # 'StatementsOfCashFlows',
        # # 'BalanceSheets'
        'IncomeTaxExpenseBenefit',
        'IncomeTaxesPaid',
        'IncomeTaxesPaidNet',
        'IncomeTaxesReceivable',
        'Liabilities',
        'LiabilitiesAndStockholdersEquity',
        'LiabilitiesCurrent',
        'LongTermDebt',
        'LongTermDebtCurrent',
        'LongTermDebtNoncurrent',
        'NetIncomeLoss',
        'OperatingLeaseCost',
        'PreferredStockDividendsAndOtherAdjustments,'
        'PreferredStockParOrStatedValuePerShare',
        'PreferredStockSharesAuthorized',
        'PreferredStockSharesIssued',
        'PreferredStockSharesOutstanding',
        'StockholdersEquity',
        'StockholdersEquityNoteStockSplitConversionRatio1',
        'StockIssued1',
        'StockIssuedDuringPeriodSharesStockOptionsExercised',
        'StockIssuedDuringPeriodValueNewIssues',
    ]

    for fact in graph_facts:

        fact_json = all_facts[fact]

        rows = []
        for currency, unit_entries in fact_json['units'].items():
            for unit_entry in unit_entries:
                fact_dict = {
                    'ticker': ticker,
                    'currency': currency,
                    'fact': fact,
                    'label': fact_json['label'],
                    'description': fact_json['description'],
                    'unit': currency
                }
                fact_dict.update(unit_entry)
                rows.append(fact_dict)
            fact_dict.update(unit_entry)
            rows.append(fact_dict)
        df_ref = pd.DataFrame(rows).dropna().set_index('end') 
        display(df_ref.head())
        fig = go.FigureWidget()
        fig.add_trace(
            go.Scatter(
                x=list(df_ref.index),
                y=list(df_ref.val),
                name=fact,
            #line=dict(color=stringToColorCode(df_copy['fact'].iloc[0])),
                showlegend=True
            )
        )

        fig.update_xaxes(title="end", rangeslider_visible=True)
        fig.update_yaxes()  # Fit y-axis to match the current slider view data)

        fig.update_layout(title=ticker + ' ' + fact,
            height=800, width=1200, showlegend=True)

        fig.show()
        
# review data
def get_cik_from_ticker(ticker):
    ticker_data = companyData[companyData['ticker'] == ticker]
    if len(ticker_data) > 0:
        cik = ticker_data['cik_str'].iloc[0]
        return cik
    else:
        return None

# get company concept data
for ticker in desired_tickers:
    try:
        cik = get_cik_from_ticker(ticker)
        process_cik_data(cik)
    except Exception as e:
        logger.error(e)
    
    
    # companyConcept = requests.get(  
    # (
    # f'https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}'
    #     f'/us-gaap/Assets.json'
    # ),
    # headers=headers
    # companyFacts = companyConcept.json()

    # )
    # # companyConcept.json().keys()
    # # companyConcept.json()['units']
    # # companyConcept.json()['units'].keys()
    # # companyConcept.json()['units']['USD']
    # # companyConcept.json()['units']['USD'][0]

    # # parse assets from single filing
    # companyConcept.json()['units']['USD'][0]['val']

    # # get all filings data 
    # assetsData = pd.DataFrame.from_dict((
    #             companyConcept.json()['units']['USD']))

    # # review data
    # assetsData.columns
    # assetsData.form

    # # get assets from 10Q forms and reset index
    # assets10Q = assetsData[assetsData.form == '10-Q']
    # assets10Q = assets10Q.reset_index(drop=True)

    # # plot 
    # assets10Q.plot(x='end', y='val')

INFO:__main__:Processing CIK: 0001355096


,ticker,currency,fact,label,description,unit,val,accn,fy,fp,form,filed,frame
end,,,,,,,,,,,,,
2008-12-31,QRTEA,USD,Assets,Assets,Sum of the carrying amounts as of the balance ...,USD,41903000000,0001047469-10-001362,2009,FY,10-K,2010-02-25,CY2008Q4I
2009-06-30,QRTEA,USD,Assets,Assets,Sum of the carrying amounts as of the balance ...,USD,42923000000,0001047469-09-007370,2009,Q2,10-Q,2009-08-07,CY2009Q2I
2009-09-30,QRTEA,USD,Assets,Assets,Sum of the carrying amounts as of the balance ...,USD,42876000000,0001047469-09-009800,2009,Q3,10-Q,2009-11-09,CY2009Q3I
2009-12-31,QRTEA,USD,Assets,Assets,Sum of the carrying amounts as of the balance ...,USD,28631000000,0001047469-11-001521,2010,FY,10-K,2011-02-28,CY2009Q4I
2010-03-31,QRTEA,USD,Assets,Assets,Sum of the carrying amounts as of the balance ...,USD,28447000000,0001047469-10-004949,2010,Q1,10-Q,2010-05-07,CY2010Q1I


,ticker,currency,fact,label,description,unit,start,val,accn,fy,fp,form,filed,frame
end,,,,,,,,,,,,,,
2017-03-31,QRTEA,USD/shares,EarningsPerShareBasic,"Earnings Per Share, Basic",The amount of net income (loss) for the period...,USD/shares,2017-01-01,0.20,0001558370-19-001344,2018,FY,10-K,2019-02-28,CY2017Q1
2017-06-30,QRTEA,USD/shares,EarningsPerShareBasic,"Earnings Per Share, Basic",The amount of net income (loss) for the period...,USD/shares,2017-04-01,0.25,0001558370-19-001344,2018,FY,10-K,2019-02-28,CY2017Q2
2017-09-30,QRTEA,USD/shares,EarningsPerShareBasic,"Earnings Per Share, Basic",The amount of net income (loss) for the period...,USD/shares,2017-07-01,0.27,0001558370-19-001344,2018,FY,10-K,2019-02-28,CY2017Q3
2017-12-31,QRTEA,USD/shares,EarningsPerShareBasic,"Earnings Per Share, Basic",The amount of net income (loss) for the period...,USD/shares,2017-10-01,2.07,0001558370-19-001344,2018,FY,10-K,2019-02-28,CY2017Q4
2018-03-31,QRTEA,USD/shares,EarningsPerShareBasic,"Earnings Per Share, Basic",The amount of net income (loss) for the period...,USD/shares,2018-01-01,0.30,0001558370-19-001344,2018,FY,10-K,2019-02-28,CY2018Q1


ERROR:__main__:'Cash'
INFO:__main__:Processing CIK: 0000320193


,ticker,currency,fact,label,description,unit,val,accn,fy,fp,form,filed,frame
end,,,,,,,,,,,,,
2008-09-27,AAPL,USD,Assets,Assets,Sum of the carrying amounts as of the balance ...,USD,36171000000,0001193125-10-238044,2010,FY,10-K,2010-10-27,CY2008Q3I
2009-06-27,AAPL,USD,Assets,Assets,Sum of the carrying amounts as of the balance ...,USD,48140000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2009Q2I
2009-09-26,AAPL,USD,Assets,Assets,Sum of the carrying amounts as of the balance ...,USD,47501000000,0001193125-11-282113,2011,FY,10-K,2011-10-26,CY2009Q3I
2009-12-26,AAPL,USD,Assets,Assets,Sum of the carrying amounts as of the balance ...,USD,53926000000,0001193125-10-012085,2010,Q1,10-Q,2010-01-25,CY2009Q4I
2010-03-27,AAPL,USD,Assets,Assets,Sum of the carrying amounts as of the balance ...,USD,57057000000,0001193125-10-088957,2010,Q2,10-Q,2010-04-21,CY2010Q1I


...

,ticker,currency,fact,label,description,unit,val,accn,fy,fp,form,filed,frame
end,,,,,,,,,,,,,
2019-12-31,GOOG,USD,LongTermDebtCurrent,"Long-term Debt, Current Maturities","Amount, after unamortized (discount) premium a...",USD,0,0001652044-21-000010,2020,FY,10-K,2021-02-03,CY2019Q4I
2020-09-30,GOOG,USD,LongTermDebtCurrent,"Long-term Debt, Current Maturities","Amount, after unamortized (discount) premium a...",USD,999000000,0001652044-20-000050,2020,Q3,10-Q,2020-10-30,CY2020Q3I
2020-12-31,GOOG,USD,LongTermDebtCurrent,"Long-term Debt, Current Maturities","Amount, after unamortized (discount) premium a...",USD,999000000,0001652044-22-000019,2021,FY,10-K,2022-02-02,CY2020Q4I
2021-03-31,GOOG,USD,LongTermDebtCurrent,"Long-term Debt, Current Maturities","Amount, after unamortized (discount) premium a...",USD,1000000000,0001652044-21-000020,2021,Q1,10-Q,2021-04-28,CY2021Q1I
2021-06-30,GOOG,USD,LongTermDebtCurrent,"Long-term Debt, Current Maturities","Amount, after unamortized (discount) premium a...",USD,0,0001652044-21-000047,2021,Q2,10-Q,2021-07-28,CY2021Q2I


,ticker,currency,fact,label,description,unit,val,accn,fy,fp,form,filed,frame
end,,,,,,,,,,,,,
2014-12-31,GOOG,USD,LongTermDebtNoncurrent,"Long-term Debt, Excluding Current Maturities",Amount after unamortized (discount) premium an...,USD,2992000000,0001652044-16-000022,2015,FY,8-K,2016-05-03,CY2014Q4I
2015-09-30,GOOG,USD,LongTermDebtNoncurrent,"Long-term Debt, Excluding Current Maturities",Amount after unamortized (discount) premium an...,USD,1994000000,0001652044-15-000005,2015,Q3,10-Q,2015-10-29,CY2015Q3I
2015-12-31,GOOG,USD,LongTermDebtNoncurrent,"Long-term Debt, Excluding Current Maturities",Amount after unamortized (discount) premium an...,USD,1995000000,0001652044-17-000008,2016,FY,10-K,2017-02-03,CY2015Q4I
2016-03-31,GOOG,USD,LongTermDebtNoncurrent,"Long-term Debt, Excluding Current Maturities",Amount after unamortized (discount) premium an...,USD,1987000000,0001652044-16-000021,2016,Q1,10-Q,2016-05-03,CY2016Q1I
2016-06-30,GOOG,USD,LongTermDebtNoncurrent,"Long-term Debt, Excluding Current Maturities",Amount after unamortized (discount) premium an...,USD,1984000000,0001652044-16-000032,2016,Q2,10-Q,2016-08-04,CY2016Q2I


,ticker,currency,fact,label,description,unit,start,val,accn,fy,fp,form,filed,frame
end,,,,,,,,,,,,,,
2013-12-31,GOOG,USD,NetIncomeLoss,Net Income (Loss) Attributable to Parent,"The portion of profit or loss for the period, ...",USD,2013-01-01,12733000000,0001652044-16-000022,2015,FY,8-K,2016-05-03,CY2013
2014-09-30,GOOG,USD,NetIncomeLoss,Net Income (Loss) Attributable to Parent,"The portion of profit or loss for the period, ...",USD,2014-07-01,2739000000,0001652044-15-000005,2015,Q3,10-Q,2015-10-29,CY2014Q3
2014-12-31,GOOG,USD,NetIncomeLoss,Net Income (Loss) Attributable to Parent,"The portion of profit or loss for the period, ...",USD,2014-01-01,14136000000,0001652044-17-000008,2016,FY,10-K,2017-02-03,CY2014
2015-03-31,GOOG,USD,NetIncomeLoss,Net Income (Loss) Attributable to Parent,"The portion of profit or loss for the period, ...",USD,2015-01-01,3515000000,0001652044-16-000021,2016,Q1,10-Q,2016-05-03,CY2015Q1
2015-06-30,GOOG,USD,NetIncomeLoss,Net Income (Loss) Attributable to Parent,"The portion of profit or loss for the period, ...",USD,2015-04-01,3931000000,0001652044-16-000032,2016,Q2,10-Q,2016-08-04,CY2015Q2


,ticker,currency,fact,label,description,unit,start,val,accn,fy,fp,form,filed,frame
end,,,,,,,,,,,,,,
2019-03-31,GOOG,USD,OperatingLeaseCost,"Operating Lease, Cost","Amount of single lease cost, calculated by all...",USD,2019-01-01,398000000,0001652044-20-000021,2020,Q1,10-Q,2020-04-29,CY2019Q1
2019-06-30,GOOG,USD,OperatingLeaseCost,"Operating Lease, Cost","Amount of single lease cost, calculated by all...",USD,2019-04-01,427000000,0001652044-20-000032,2020,Q2,10-Q,2020-07-31,CY2019Q2
2019-09-30,GOOG,USD,OperatingLeaseCost,"Operating Lease, Cost","Amount of single lease cost, calculated by all...",USD,2019-07-01,477000000,0001652044-20-000050,2020,Q3,10-Q,2020-10-30,CY2019Q3
2019-12-31,GOOG,USD,OperatingLeaseCost,"Operating Lease, Cost","Amount of single lease cost, calculated by all...",USD,2019-01-01,1820000000,0001652044-21-000010,2020,FY,10-K,2021-02-03,CY2019
2020-03-31,GOOG,USD,OperatingLeaseCost,"Operating Lease, Cost","Amount of single lease cost, calculated by all...",USD,2020-01-01,523000000,0001652044-20-000021,2020,Q1,10-Q,2020-04-29,CY2020Q1


ERROR:__main__:'PreferredStockDividendsAndOtherAdjustments,PreferredStockParOrStatedValuePerShare'
INFO:__main__:Processing CIK: None
ERROR:__main__:Expecting value: line 1 column 1 (char 0)
INFO:__main__:Processing CIK: None
ERROR:__main__:Expecting value: line 1 column 1 (char 0)
INFO:__main__:Processing CIK: None
ERROR:__main__:Expecting value: line 1 column 1 (char 0)
INFO:__main__:Processing CIK: None
ERROR:__main__:Expecting value: line 1 column 1 (char 0)
INFO:__main__:Processing CIK: None
ERROR:__main__:Expecting value: line 1 column 1 (char 0)
INFO:__main__:Processing CIK: None
ERROR:__main__:Expecting value: line 1 column 1 (char 0)
INFO:__main__:Processing CIK: None
ERROR:__main__:Expecting value: line 1 column 1 (char 0)
INFO:__main__:Processing CIK: None
ERROR:__main__:Expecting value: line 1 column 1 (char 0)
INFO:__main__:Processing CIK: 0001041130
ERROR:__main__:Expecting value: line 1 column 1 (char 0)
INFO:__main__:Processing CIK: 0001067839
ERROR:__main__:Expecting v